In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os

# Define the Generator class
class Generator(nn.Module):
    def __init__(self, latent_dim, image_size):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, image_size),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img

# Define the Discriminator class
class Discriminator(nn.Module):
    def __init__(self, image_size):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(image_size, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity

# Training parameters
latent_dim = 100
image_size = 784  # 28x28 for MNIST images
batch_size = 64
epochs = 50  # Reduced for faster training
lr = 0.0002

# Initialize models
generator = Generator(latent_dim, image_size)
discriminator = Discriminator(image_size)

# Loss function and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [02:20<00:00, 70.4kB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 105kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:54<00:00, 30.0kB/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.15MB/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [ ]:

# Training loop
os.makedirs("images", exist_ok=True)
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real_imgs = imgs.view(imgs.size(0), -1)
        valid = torch.ones((imgs.size(0), 1))
        fake = torch.zeros((imgs.size(0), 1))

        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn((imgs.size(0), latent_dim))
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")

    # Save generated images
    with torch.no_grad():
        z = torch.randn(16, latent_dim)
        sample_imgs = generator(z).view(-1, 1, 28, 28)
        grid_img = torch.cat([sample_imgs[i] for i in range(16)], dim=2)
        plt.imshow(grid_img.permute(1, 2, 0).squeeze(), cmap="gray")
        plt.axis("off")
        plt.savefig(f"images/epoch_{epoch}.png")
        plt.close()

[Epoch 0/50] [Batch 0/938] [D loss: 0.7698] [G loss: 0.6863]
[Epoch 0/50] [Batch 1/938] [D loss: 0.6254] [G loss: 0.6737]
[Epoch 0/50] [Batch 2/938] [D loss: 0.5399] [G loss: 0.6515]
[Epoch 0/50] [Batch 3/938] [D loss: 0.4825] [G loss: 0.6399]
[Epoch 0/50] [Batch 4/938] [D loss: 0.4525] [G loss: 0.6204]
[Epoch 0/50] [Batch 5/938] [D loss: 0.4336] [G loss: 0.6122]
[Epoch 0/50] [Batch 6/938] [D loss: 0.4315] [G loss: 0.5949]
[Epoch 0/50] [Batch 7/938] [D loss: 0.4285] [G loss: 0.5836]
[Epoch 0/50] [Batch 8/938] [D loss: 0.4314] [G loss: 0.5692]
[Epoch 0/50] [Batch 9/938] [D loss: 0.4308] [G loss: 0.5662]
[Epoch 0/50] [Batch 10/938] [D loss: 0.4393] [G loss: 0.5496]
[Epoch 0/50] [Batch 11/938] [D loss: 0.4440] [G loss: 0.5402]
[Epoch 0/50] [Batch 12/938] [D loss: 0.4507] [G loss: 0.5309]
[Epoch 0/50] [Batch 13/938] [D loss: 0.4569] [G loss: 0.5195]
[Epoch 0/50] [Batch 14/938] [D loss: 0.4621] [G loss: 0.5128]
[Epoch 0/50] [Batch 15/938] [D loss: 0.4649] [G loss: 0.5083]
[Epoch 0/50] [Batc